In [2]:
import optparse
import pandas as pd
import numpy as np
from sklearn import preprocessing
import tensorflow as tf
import os, time, sys, sklearn
from sklearn.externals import joblib
from rnn_functions import *
import time
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
import keras
import os
from sklearn.utils import class_weight
os.environ["TF_MIN_GPU_MULTIPROCESSOR_COUNT"] = "4"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

Using TensorFlow backend.


In [3]:
DATA_PARAMS = dict()        
DATA_PARAMS["TARGET_TO_PREDICT"] = "AUDUSD"
DATA_PARAMS["FUTURE_PERIOD_PREDICT"] = 5
DATA_PARAMS["SEQ_LEN"] = 60
DATA_PARAMS["TARGET_THRESHOLD"] = 0.001

dirdata = os.path.join('cnn',DATA_PARAMS["TARGET_TO_PREDICT"])
init_dir(dirdata)

In [4]:
assetdir = os.path.join("cnn", DATA_PARAMS["TARGET_TO_PREDICT"], "data")
files = os.listdir(assetdir)
vars_name = [j.split(".")[0] for j in files]

j = 0
for j in range(len(vars_name)):
    exec(vars_name[j] + ' = joblib.load(os.path.join(assetdir, "' + vars_name[j] + '.pkl"))' )

In [ ]:
EPOCHS = 200
logs_folder = assetdir = os.path.join("cnn", DATA_PARAMS["TARGET_TO_PREDICT"], "logs")
models_folder = assetdir = os.path.join("cnn", DATA_PARAMS["TARGET_TO_PREDICT"], "models")
init_dir(logs_folder)
init_dir(models_folder)
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(train_y),
                                                 train_y)


In [ ]:
t0 = time.time()

model = Sequential()
model.add(Conv2D(128, kernel_size=(15,1), activation='relu', input_shape=(train_x_shape[1],train_x_shape[2],1)))
model.add(Conv2D(16, kernel_size=(2,1), activation='relu'))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

adm = keras.optimizers.Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=None, amsgrad=False, decay = 1e-6)
model.compile(optimizer=adm, loss='binary_crossentropy', metrics=['accuracy', precision])
                                                                  
tensorboard = TensorBoard(log_dir=logs_folder)
filepath = "CNN-{epoch:03d}-{val_loss:.4f}-{val_acc:.4f}"
checkpoint = tf.keras.callbacks.ModelCheckpoint("{}/{}.model".format(models_folder, filepath),
                                                   monitor='val_loss',
                                                   verbose=1,
                                                   save_best_only=True,
                                                   save_weights_only=False,
                                                   mode='min')
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=int(EPOCHS/3), verbose=0, mode='min')
        
history = model.fit(train_x, train_y, batch_size=256, validation_data=(val_x, val_y), epochs=EPOCHS, class_weight=class_weights, callbacks=[tensorboard, checkpoint, earlystopping])

t1 = time.time()


In [ ]:
print(t1-t0)

In [ ]:
# x,y = train_x, train_y
# x,y = val_x, val_y
# x,y = test_x, test_y

# y_true = []
# y_score = []
# for j in range(1000):
#     t2,_ = reshape2(x[j:(j+1)])
#     ypred = model.predict(t2)
#     ypred = np.array(ypred).flatten()[0]
#     ytrue = y[j]
#     y_true.append(ytrue)
#     y_score.append(ypred)

def perf(x,y, t = 0.5):
    xx, _ = reshape2(x)
    ypred = model.predict(xx)
    ypred = np.array(ypred).flatten()
    ypred_bin = 1*(np.array(ypred) > t)
    ytrue = y


    # print(y_true)
    # print(y_score)
    print("n:", len(y))
    print("true: prop:", np.mean(ytrue), "n1:", np.sum(ytrue))
    print("pred: prop:", np.mean(ypred_bin), "n1:", np.sum(ypred_bin))
    cm = sklearn.metrics.confusion_matrix(ytrue, ypred_bin)
    prec = sklearn.metrics.precision_score(ytrue, ypred_bin)
    auc = sklearn.metrics.roc_auc_score(ytrue, np.array(ypred))
    print("cm:\n",cm)
    print("prec:",prec)
    print("auc:", auc)

In [ ]:
perf(train_x, train_y)

In [ ]:
perf(val_x, val_y)


In [ ]:
perf(test_x, test_y, t = 0.50)